In [2]:
import pandas as pd
from scipy import sparse

In [4]:
ratings = pd.read_csv('../assets/datasets/music_workflow.csv')
ratings = ratings.drop(['task_type','index','urgency','importance','keywords','id','career_stage','tools_required'],axis=1)
print(ratings.shape)
ratings.head()

(100, 3)


,userID,name,rating
0,145,Plan music release campaign,8.7
1,146,Plan music release campaign,8.2
2,147,Plan music release campaign,7.9
3,148,Plan music release campaign,8.8
4,149,Plan music release campaign,9.1


In [5]:
userRatings = ratings.pivot_table(index=['userID'],columns=['name'],values='rating')
userRatings.head()
print("Before: ",userRatings.shape)
userRatings = userRatings.dropna(thresh=5, axis=1).fillna(0,axis=1)
#userRatings.fillna(0, inplace=True)
print("After: ",userRatings.shape)

userRatings.head()

Before:  (100, 20)
After:  (100, 8)


name,Analyze Spotify statistics,Create TikTok content calendar,Design Instagram Reels strategy,Engage with fan comments,Plan music release campaign,Plan release day promotion,Set up Discord community,Submit for playlist consideration
userID,,,,,,,,
145,0.0,0.0,0.0,0.0,8.7,0.0,0.0,0.0
146,0.0,0.0,0.0,0.0,8.2,0.0,0.0,0.0
147,0.0,0.0,0.0,0.0,7.9,0.0,0.0,0.0
148,0.0,0.0,0.0,0.0,8.8,0.0,0.0,0.0
149,0.0,0.0,0.0,0.0,9.1,0.0,0.0,0.0


In [18]:
corrMatrix = userRatings.corr(method='pearson')
corrMatrix.head(100)

name,Analyze Spotify statistics,Create TikTok content calendar,Design Instagram Reels strategy,Engage with fan comments,Plan music release campaign,Plan release day promotion,Set up Discord community,Submit for playlist consideration
name,,,,,,,,
Analyze Spotify statistics,1.000000,-0.067625,-0.067630,-0.062910,-0.076402,-0.072127,-0.072126,-0.067633
Create TikTok content calendar,-0.067625,1.000000,-0.086921,-0.080855,-0.098196,-0.092701,-0.092700,-0.086925
Design Instagram Reels strategy,-0.067630,-0.086921,1.000000,-0.080860,-0.098202,-0.092707,-0.092706,-0.086931
Engage with fan comments,-0.062910,-0.080855,-0.080860,1.000000,-0.091349,-0.086237,-0.086236,-0.080864
Plan music release campaign,-0.076402,-0.098196,-0.098202,-0.091349,1.000000,-0.104733,-0.104731,-0.098207
Plan release day promotion,-0.072127,-0.092701,-0.092707,-0.086237,-0.104733,1.000000,-0.098871,-0.092712
Set up Discord community,-0.072126,-0.092700,-0.092706,-0.086236,-0.104731,-0.098871,1.000000,-0.092711
Submit for playlist consideration,-0.067633,-0.086925,-0.086931,-0.080864,-0.098207,-0.092712,-0.092711,1.000000


In [40]:
def get_similar(session_name, rating):
    similar_ratings = corrMatrix[session_name]*(rating-2.5)
    similar_ratings = pd.DataFrame({
        'Session': corrMatrix[session_name].index,
        'Similarity': similar_ratings.values
    })
    #similar_ratings = pd.concat([corrMatrix[session_name], similar_ratings])
    return similar_ratings

In [51]:
content_creator = [
    ("Create TikTok content calendar", 8.9),
    ("Plan music release campaign", 8.7),
    ("Design Instagram Reels strategy", 8.8),
    ("Engage with fan comments", 7.8),
    ("Submit for playlist consideration", 8.9),
    ("Plan release day promotion", 9.0)
]
similar_sessions = pd.DataFrame()
for session,rating in content_creator:
    similar_sessions = pd.concat([similar_sessions, get_similar(session,rating)], ignore_index=True)

# print(similar_sessions.head(10))
similar_sessions.sort_values(by='Similarity', ascending=False).head(20)

,Session,Similarity
45,Plan release day promotion,6.500000
1,Create TikTok content calendar,6.400000
39,Submit for playlist consideration,6.400000
18,Design Instagram Reels strategy,6.300000
12,Plan music release campaign,6.200000
27,Engage with fan comments,5.300000
24,Analyze Spotify statistics,-0.333422
16,Analyze Spotify statistics,-0.426066
25,Create TikTok content calendar,-0.428530
26,Design Instagram Reels strategy,-0.428559


In [47]:
community_manager = [
    ("Set up Discord community", 9.2),
    ("Engage with fan comments", 9.1),
]

similar_sessions = pd.DataFrame()
for session,rating in community_manager:
    similar_sessions = pd.concat([similar_sessions, get_similar(session,rating)], ignore_index=True)

similar_sessions.head(10)
similar_sessions.sort_values(by='Similarity', ascending=False).head(20)

,Session,Similarity
6,Set up Discord community,6.700000
11,Engage with fan comments,6.600000
8,Analyze Spotify statistics,-0.415204
0,Analyze Spotify statistics,-0.483243
9,Create TikTok content calendar,-0.533641
10,Design Instagram Reels strategy,-0.533677
15,Submit for playlist consideration,-0.533704
14,Set up Discord community,-0.569158
13,Plan release day promotion,-0.569167
3,Engage with fan comments,-0.577781
